In [1]:
import copy

from Game import * 
import params

In [2]:
class Node:
    def __init__(self):
        self.edges = []
        

In [3]:
class Edge:
    def __init__(self,x,y):
        self.N = 0
        self.W = 0.0
        self.Q = 0.0
        self.P = 0.0
        self.act_x = x
        self.act_y = y
        
        self.result_node = None
        
        
        

In [4]:
def reach_leaf_node_and_update_edges(g, root_node):
    pass

In [5]:
def extend_leaf_node(g, leaf_node):
    pass

In [6]:
def MCTS(game, root_node = None): 
    if root_node == None:
        root_node = Node()
            
    for i in range(params.MTCS_NR):
        g = copy.deep_copy(game)
        g,leaf_node = reach_leaf_node_and_update_edges(g, root_node)
        extend_leaf_node(g, leaf_node)
    
    #select node
    #...
    #return x,y,next_node
